In [6]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [7]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import time
import math

import warnings
warnings.filterwarnings('ignore')

In [8]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [9]:
df = pd.read_excel('data_mapped.xlsx')
df

,Column1,업소명,한옥 여부,업소위치,전화번호,지번주소,도로명주소,객실수,룸타입수,위도,경도,ID
0,67,옹게스트하우스,1,덕진구 진북동,010-****-****,전라북도 전주시 덕진구 진북동 1150-20,전라북도 전주시 덕진구 숲정이3길 5,1,1,35.829913,127.13177,NaN
1,136,달달잠,1,완산구 교동,010-****-****,전라북도 전주시 완산구 교동 960-7,전라북도 전주시 완산구 간납대2길 28,6,4,35.816883,127.158973,ChIJAeMcsXwjcDURdygDt0YHdhE
2,137,이가한옥,1,완산구 풍남동,010-****-****,전라북도 전주시 완산구 풍남동3가 7-13,전라북도 전주시 완산구 간납로 8,12,3,35.818227,127.160056,ChIJe1xCB2MjcDURmuclTFSdun8
3,138,생황토방,1,완산구 교동,010-****-****,전라북도 전주시 완산구 교동 79-46,전라북도 전주시 완산구 경기전길 125-16,4,3,35.813648,127.151208,Ei1SNTcyK0dNLCBKZW9uanUtc2ksIEplb2xsYWJ1ay1kby...
4,139,가원당,1,완산구 교동,010-****-****,전라북도 전주시 완산구 교동 79-44,전라북도 전주시 완산구 경기전길 125-9,11,5,35.813648,127.151208,ChIJoYFRbJ4kcDURhZXM-krhf04
...,...,...,...,...,...,...,...,...,...,...,...,...
466,466,호텔모아,0,완산구 효자동 3가,063-221-9501,전라북도 전주시 완산구 효자동3가 1537-4,전라북도 전주시 완산구 홍산북로 80,36,5,35.817497,127.109023,NaN
467,467,호텔아이,0,완산구 효자동,063-237-5590,전라북도 전주시 완산구 효자동3가 1537-5,전라북도 전주시 완산구 홍산북로 84,48,3,35.817497,127.109023,NaN
468,468,모텔쿡,0,완산구 효자동,063-236-0036,전라북도 전주시 완산구 효자동3가 1537-8,전라북도 전주시 완산구 홍산북로 84-10,16,2,35.817497,127.109023,NaN
469,469,호텔유,0,완산구 효자동 3가,063-221-5510,전라북도 전주시 완산구 효자동3가 1537-9,전라북도 전주시 완산구 홍산북로 84-12,24,1,35.817497,127.109023,NaN


In [5]:
df = df[df['한옥 여부'] == 1]

In [6]:
df['업소명']

0      옹게스트하우스
1          달달잠
2         이가한옥
3         생황토방
4          가원당
        ...   
198       햇살가득
199       귀거래사
200       온새미로
201       나리다솜
202         다하
Name: 업소명, Length: 203, dtype: object

In [7]:
name = []
data = []

In [8]:
def extract_review():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    """//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li[1]/div[4]/a 
    //*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li[3]/div[4]/a
    //*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li[1]/div[4]/a
    //*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li[1]/div[3]/a
    //*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li[4]/div[4]/a"""
    
    rev = []
    for i in range(1, 11): # 더보기 누르지 않은 상태로 최대 10개
        try:  # 사진 없는 후기는 div 3번째에 텍스트 위치
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
            time.sleep(2)
            comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
            rev.append(comment)
        except: # 사진 있는 후기는 div 4번째에 텍스트가 위치
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li['+str(i)+']/div[4]/a').send_keys(Keys.ENTER)
            time.sleep(2)
            comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li['+str(i)+']/div[4]/a/span').text  # 리뷰
            rev.append(comment)
    return rev

In [9]:
# 크롬 드라이버 실행
driver = webdriver.Chrome(ChromeDriverManager().install())

for i in df['업소명']:
    url = "https://map.naver.com/v5/search/전주 "+str(i)+'/place' 
    driver.get(url)
    time.sleep(8)
    
    # 숙소 이름
    name.append(i)
    print(i)
    
    try: 
        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
        time.sleep(3) 
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        lists = soup.select('#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm.f7aZ0 > div.dAsGb > span')
        if len(lists) > 2: # 별점/방문자 리뷰/블로그 리뷰 순일때 방문자리뷰는 두번째에 위치=span[2]
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a').send_keys(Keys.ENTER) # 방문자 리뷰
            time.sleep(3)

        else: # 방문자 리뷰/블로그 리뷰 순일때 방문자리뷰는 첫번째에 위치=span[1]
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a').send_keys(Keys.ENTER) # 방문자 리뷰
            time.sleep(3)
        
        review = extract_review() # 리뷰 추출 함수
        data.append(review)
        # print(review)
    
    except:
        data.append(' ')
        # print(' ')

 
['최고의 숙소였어요!:D 어린이날연휴에 2층독채에 어른6명 아이5명 총11명이 함께했었는데, 너무 편안하게 잘 쉬다 왔습니다. 비가 많이 와서 외부 여행지보다 숙소에 있던 시간이 많았었는데, 너무 친절하신 사장님의 세심한 배려들(궁금하시다면 꼭 가보세요!)덕에 기분 좋게 잘 보냈습니다. 각 방에 화장실이 있어서 더 편했고, 침구나 수건도 깨끗해서 잘 사용할 수 있었습니다. 기분 좋은 여행을 보낼 수 있게 해주셔서 감사드립니다 ~ 강추 입니다! 달달잠!', '아빠 생신이여서 가족여행을 다녀왔어요.\n총 11명이 이용했는데 방이 많아서\n나눠서 이용하기에도 좋았고\n각 방마다 화장실이 있어서\n씻을때도 유용했어요.\n\n체크인 전과 체크아웃후에도\n친절하게 연락주시고 신경써주셔서\n정말 감사했습니다.\n한옥마을도 가까워서 맛집 투어도 잘하고\n돌아왔어요^^\n\n숙소 앞에 바로 주차도 할 수 있어서\n편하게 이용했습니다.', '10인이 이용하였는데 전부 만족했던 숙소입니다\n한옥마을에서 좀 걸어가야되긴한데\n오히려 주차 편히 해놓고 실컷 즐기다와도 되네요\n한옥마을 입구서 5분정도 걸은거같아요~\n그리고 주인아주머니께서 넘나 친절하셨어요~\n그날 눈오고 추운날인데 입실하기도 전에 방 데워졌다고~언능 와서 쉬라고..\n그리고 각방마다 화장실이 있어 단체팀이 이용하기 넘나좋은 숙소였어요\n각 방이랑 화장실마다 드라이기 및 샴푸,폼클렌징 등 다 있었구요\n수건도 삶아놓은것처럼 깨끗하고 부족함없이 비치해놓으셨더라구요\n침구류도 좋아서 다들 이불 만족했다고..방 자글자글하니 푹 잠들고..일어날 때 개운했다고 그랬어요~^^\n담에 전주갈 때 꼭 다시 이용하고 싶은 숙소였답니다', '2박3일동안 너무 잘지내다 왔어요❤️\n사장님 너무 친절하시고 입실시간맞춰 주차자리 마련해주시고 이것저것 신경써주신게 딱 느껴졌어요\n친정식구들 어른7명 아이4명이 좁지 않게 보내고 왔어요~^^\n다음에 전주가게되면 재방문의사 100%입니다👍', '독채지만 방이 다섯개다보니 잠자리가 편하더라

 
 
 
['한옥마을안에있는줄알았는데 상세확인을못한불찰이죠 한옥마을에서 차로10분거리에 있어 인터넷검색하니 저녁먹을만한곳도없고 한옥마을에서 놀다가 저녁먹고 간단한안주거리 사서 숙소에서먹었네요 숙소는\n깻끗하고 이불도넉넉하고 입실하니까 보일러가켜저있네요 날씨는 안추워는데 좀더워네요 한옥마을안에없어서좀아쉬워네요', '한옥마을 초입에 있어서 이동이 편리하고 사장님이 완전 친절하세요 주차도 자리 잡아주시고 중간에 이동해도 자리 맡아주셔서 편하게 이용했어요 방도 조용했고 여자 3명이서 이용했는데 불편함 없었습니다', '하루 저렴히 자기에 좋은 숙소였습니다. 예민하시지 않다면 적당히 깔끔하고, 화장실은 잘 되어 있습니다. 불편한 점이 있을 땐 계속 연락하시면 잘 대응해 주십니다.', '사장님도 너무 친절하시고 너무 잘쉬다왔어요', '다른방은 한옥으로 된 본채에 있고, 이 방은 차로 10분쯤 떨어진 곳에 별채처럼 있는 "빌라"입니다. "빌라 쓰리룸형" 타입을 선택하실때는 한옥스테이 생각하면 안되요~ \n가기 전에는 정보랑 후기가 없어서 걱정했는데, 결과적으로 저는 대만족했습니다.\n일단 8인까지 추가금이 없어요. 그리고 도착하기 전 미리 연락드리니 한옥마을 내 본채 앞에 주차하고 구경다닐 수 있게 해 주셨고, 오후 느즈막히 숙소(빌라)로 가니 미리 보일러 틀어두셨더라고요. 한옥은 음식 반입금지인 곳이 꽤 많던데 여기는 취사도 가능하고요.(전자렌지,라면포트,전기포트,냉장고,웍,후라이팬 있어요. 전기밥솥 없는게 유일하게 아쉬웠어요.)\n밤에 내린 비로 다음날 돌아다니기 추워 오후일정까지 시간이 애매했는데 체크아웃도 미뤄주셔서 너무 감사했습니다.', '따뜻한 온돌에 편안하게 쉬다가요 사장님 친절하시고 좋아요!!', '칫솔 챙겨야함. 취사 불가 (냉장고, 전기포트,전자레인지 없음)', '두가족 잘 쉬었다가 갑니다 전주 갈시에 또 이용할께요~!!', '잠만 자고 나올생각으로 한옥마을에서 가까운 곳을 선택해서 갔어요\n가격대비 생각하시고 가시면 좋아요\n잘쉬었다갑니다.', '가성비

KeyboardInterrupt: 

In [10]:
data

[' ',
 ['최고의 숙소였어요!:D 어린이날연휴에 2층독채에 어른6명 아이5명 총11명이 함께했었는데, 너무 편안하게 잘 쉬다 왔습니다. 비가 많이 와서 외부 여행지보다 숙소에 있던 시간이 많았었는데, 너무 친절하신 사장님의 세심한 배려들(궁금하시다면 꼭 가보세요!)덕에 기분 좋게 잘 보냈습니다. 각 방에 화장실이 있어서 더 편했고, 침구나 수건도 깨끗해서 잘 사용할 수 있었습니다. 기분 좋은 여행을 보낼 수 있게 해주셔서 감사드립니다 ~ 강추 입니다! 달달잠!',
  '아빠 생신이여서 가족여행을 다녀왔어요.\n총 11명이 이용했는데 방이 많아서\n나눠서 이용하기에도 좋았고\n각 방마다 화장실이 있어서\n씻을때도 유용했어요.\n\n체크인 전과 체크아웃후에도\n친절하게 연락주시고 신경써주셔서\n정말 감사했습니다.\n한옥마을도 가까워서 맛집 투어도 잘하고\n돌아왔어요^^\n\n숙소 앞에 바로 주차도 할 수 있어서\n편하게 이용했습니다.',
  '10인이 이용하였는데 전부 만족했던 숙소입니다\n한옥마을에서 좀 걸어가야되긴한데\n오히려 주차 편히 해놓고 실컷 즐기다와도 되네요\n한옥마을 입구서 5분정도 걸은거같아요~\n그리고 주인아주머니께서 넘나 친절하셨어요~\n그날 눈오고 추운날인데 입실하기도 전에 방 데워졌다고~언능 와서 쉬라고..\n그리고 각방마다 화장실이 있어 단체팀이 이용하기 넘나좋은 숙소였어요\n각 방이랑 화장실마다 드라이기 및 샴푸,폼클렌징 등 다 있었구요\n수건도 삶아놓은것처럼 깨끗하고 부족함없이 비치해놓으셨더라구요\n침구류도 좋아서 다들 이불 만족했다고..방 자글자글하니 푹 잠들고..일어날 때 개운했다고 그랬어요~^^\n담에 전주갈 때 꼭 다시 이용하고 싶은 숙소였답니다',
  '2박3일동안 너무 잘지내다 왔어요❤️\n사장님 너무 친절하시고 입실시간맞춰 주차자리 마련해주시고 이것저것 신경써주신게 딱 느껴졌어요\n친정식구들 어른7명 아이4명이 좁지 않게 보내고 왔어요~^^\n다음에 전주가게되면 재방문의사 100%입니다👍',
  '독채지만 방이 다섯개

In [11]:
temp = []
for i in data:
    if i != ' ':
        temp.append(i)

len(temp) # 리뷰 있는 숙소 개수

In [14]:
pd.DataFrame({'name':name, 'review':data})

,review
0,
1,[최고의 숙소였어요!:D 어린이날연휴에 2층독채에 어른6명 아이5명 총11명이 함께...
2,[주말에 엄청난 인파로 한옥마을 주차장에 주차하기가 쉽지 않았는데 먼저 주차만 해도...
3,"[침구류가 깔끔하고, 난방이 잘되어 황토방에서 아늑하게 지냈습니다.\n한옥마을에 위..."
4,
5,
6,[너무 친절하시고 주변에 맛집과 가볼만한곳 정보들도 챙겨주시고 진짜 편히 잘 쉬다왔...
7,[고양이때문에 선택했다고 해도 과언이 아닌 숙소!!\n예약전에 사장님께 전화를 많이...
8,[하룻밤의 머무름에 더해진 갓만들어서 주산 떡과 유자차의 따뜻함이 약간의 불편함과 ...
9,


In [11]:
# 크롬 드라이버 실행
driver = webdriver.Chrome(ChromeDriverManager(version="114.0.5735.90").install())

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.30M/6.30M [00:03<00:00, 1.82MB/s]


In [ ]:
url = "https://map.naver.com/v5/search/전주 달달잠/place"
driver.get(url)

In [ ]:
driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]'))
time.sleep(3) 

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
lists = soup.select('#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm.f7aZ0 > div.dAsGb > span')

In [ ]:
len(lists)

In [ ]:
if len(lists) > 2:
    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a').send_keys(Keys.ENTER) # 방문자 리뷰
    time.sleep(3)
    
else:
    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a').send_keys(Keys.ENTER) # 방문자 리뷰
    time.sleep(3)

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
rev = []

In [ ]:
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li[1]/div[3]/a').send_keys(Keys.ENTER)

In [ ]:
comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li[1]/div[3]/a/span').text  # 리뷰

In [ ]:
comment